In [ ]:
import os
from time import sleep
import pandas as pd

# from selenium.webdriver.common.keys import Keys
from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support.ui import Select
# from concurrent.futures import ThreadPoolExecutor

opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])
opt.add_argument("--headless")
# opt.add_argument("--lang=en")
# import chromedriver_autoinstaller
# chromedriver_autoinstaller.install()



In [ ]:
WRITE_FLAG=True

In [ ]:
root_data_folder="../data/raw/campsandclinincs/"
output_folder_name="mass_elite"
output_file_name="mass_elite"
output_file_path = os.path.join(root_data_folder,output_folder_name)
if (not os.path.exists(output_file_path)):
    os.makedirs(output_file_path)


In [ ]:
base_link="https://www.masselite.com/team/college-camps-and-clinics/"

In [ ]:
driver = webdriver.Chrome(options=opt)
print(f"getting {base_link}")
driver.get(base_link)
sleep(2)

In [25]:
def load_webpage(
        weblink:str,
        driver_obj:object,
        sleep_time=2,
        increment=1,
        limit=10
    ):
    """to load webpage recursively"""
    print(f"getting {weblink}")
    driver_obj.get(weblink)
    print('loading webspage ... '+str(sleep_time+increment))
    if limit < sleep_time+increment:
        return False
    sleep(sleep_time)
    # check if page is loaded
    if not EC.presence_of_element_located('.//table[@class="tablepress tablepress-id-47 dataTable no-footer"]'):
        return load_webpage(weblink=weblink,driver_obj=driver_obj,sleep_time=sleep_time+increment,increment=increment,limit=limit)
    return True

In [22]:
try:
    print("scrapping all rows please wait....")
    table = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, './/table[@class="tablepress tablepress-id-47 dataTable no-footer"]'))
            )
    df_main = pd.read_html(table.get_attribute("outerHTML"))[0]
    all_links=[]
    rows = table.find_elements(by=By.XPATH, value=".//tr")

    # Iterate through each row
    for row in rows:
        # Extract the cells of the current row
        cells = row.find_elements(by=By.XPATH, value=".//td")

        # Check if the row has cells
        if cells:
            # Get the last cell of the row
            last_cell = cells[-1]

            # Check if the last cell contains a link
            link = last_cell.find_element(by=By.XPATH, value=".//a[@href]").get_attribute("href")

            all_links.append(link)
    df_main['CAMP LINK']=all_links
    # convert list of dicts to data frame
except Exception as e:
    print("couldnt scrape the data\n exception occured" )
    print(e)


scrapping all rows please wait....


In [23]:
driver.close()

In [24]:
if WRITE_FLAG:
    if df_main.shape != '(0,0)':
        df_main.to_csv(f"{output_file_path}/{output_file_name}.csv", index=False)
        print(f'completed,\n{df_main.shape} record found, \nFile saved successfully...')
    else:
        print('data frame is empty')

completed,
(62, 5) record found, 
File saved successfully...
